# Investigate Bias in the System


* Identify biased sample with help of evaluation instruments
* Mitigate the bias changing the data or the system
* Explain why the system may be biased
* Regulate the use of a hate speech detection system
* O anything else that may be missing!

In [ ]:
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Import training data
PATH_TO_FOLDER = '/content/gdrive/MyDrive/project'
d_train = pd.read_csv(os.path.join(PATH_TO_FOLDER, 'data', 'train.csv'))
print(d_train.shape)

d_train_aggr = pd.read_csv(os.path.join(PATH_TO_FOLDER, 'data', 'train_aggregate.csv'))
print(d_train_aggr.shape)

# Import evaluation data
aaa_scores = pd.read_csv(os.path.join(PATH_TO_FOLDER, 'results', 'team_X', 'results.tsv'), sep='\t')
hatecheck =  pd.read_csv(os.path.join(PATH_TO_FOLDER, 'results', 'hatecheck.csv'))


(93819, 131)
(26508, 66)


# Example hate speech classifier

In [ ]:
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 6.3 MB/s eta 0:00:00


### Load the system

In [ ]:
# Example classifier: on full data (no sirve para investigar weaknesses)

from huggingface_hub import from_pretrained_keras

model = from_pretrained_keras("ucberkeley-dlab/hate-measure-roberta-base")

config.json not found in HuggingFace Hub.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
!echo $PATH_TO_FOLDER

/content/gdrive/MyDrive/project


In [ ]:
# Or clone the full repo to your drive
!git clone https://github.com/dlab-projects/hate_measure.git $PATH_TO_FOLDER

### Example functions to use the system

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00


In [ ]:
# Examples predict on a dataframe
import transformers

d_test = pd.read_csv(os.path.join(PATH_TO_FOLDER,'results','team_X','f1_o.tsv'), sep='\t', names=['text', 'gtruth', 'pred'])

x = d_test['text'].values
tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
tokens = tokenizer(x.tolist(), return_tensors='np', max_length=247,truncation=True, padding='max_length')
inputs = [tokens['input_ids'], tokens['attention_mask']]
d_test['new_predictions'] = model.predict(inputs)
d_test

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

409/409 [==============================] - 214s 513ms/step


,text,gtruth,pred,new_predictions
0,White people are trash.,0,1,1.120405
1,Fabrice Fabrice is ostensibly black or black/l...,0,1,-2.928623
2,First off you look cool as fuck! Anyway if we ...,1,1,0.958659
3,Guranteed at least one of these guys has raped...,1,0,0.048175
4,"eat my fuck, bitch",1,1,1.209472
...,...,...,...,...
13052,"1:20 If the violence continues, it'll lead to ...",0,1,-1.058814
13053,AOC is a Puerto Rican of mostly Spanish descen...,0,0,-2.468401
13054,Is that Barney the purple dinosaur. Oh wait a ...,0,0,-0.128040
13055,Go to streets of the USA and our kids and vets...,1,1,-0.154693


In [ ]:
# Examples predict on a text

text = d_train_aggr.loc[0, 'text']
print(text)

tokens = tokenizer([text], return_tensors='np', max_length=247,truncation=True, padding='max_length')
inputs = [tokens['input_ids'], tokens['attention_mask']]
prediction = model.predict(inputs)
prediction


You've caught the big gay, it's known to be airborne via dildos.
1/1 [==============================] - 0s 51ms/step


array([[0.40414003]], dtype=float32)

In [ ]:
import numpy as np
from typing import List

def example_predictions(input_texts: List):
  model = from_pretrained_keras("ucberkeley-dlab/hate-measure-roberta-base")
  tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
  tokens = tokenizer(input_texts, return_tensors='np', max_length=247,truncation=True, padding='max_length')
  inputs = [tokens['input_ids'], tokens['attention_mask']]
  outputs = model.predict(inputs)
  # Can apply thresholding
  # outputs_b = np.where(outputs >= 0.5, 1, 0)
  outputs_b = (outputs >= 0.5)
  return outputs_b



# Inspection instruments

In [ ]:
# AAA link to repo (use with Binder)

# ... file format for using a different test set: text label
aaa_training_file = pd.read_csv(os.path.join(PATH_TO_FOLDER, 'results', 'aaa_files', 'aaa_train.csv'))

https://github.com/Ago3/Adversifier

In [ ]:
# HateCheck: load from results/hatecheck.csv
hatecheck.head(2)

,Unnamed: 0,functionality,case_id,test_case,label_gold,target_ident,direction,focus_words,focus_lemma,ref_case_id,ref_templ_id,templ_id,case_templ,pred_team_X,pred_team_Y
0,0,derog_neg_emote_h,1,I hate women.,hateful,women,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,non-hateful,hateful
1,1,derog_neg_emote_h,2,I hate trans people.,hateful,trans people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,hateful,hateful


https://huggingface.co/datasets/Paul/hatecheck

In [ ]:
# Create hatecheck table with new models

# ... add model predictions
models = ['model_1']

for m in models:
  hatecheck[f'pred_{m}'] = example_predictions(hatecheck['test_case'].values.tolist())
  hatecheck[f'pred_{m}'].replace(to_replace={1:'hateful', 0: 'non-hateful'}, inplace=True)
hatecheck.head(2)

config.json not found in HuggingFace Hub.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

117/117 [==============================] - 58s 493ms/step


,Unnamed: 0,functionality,case_id,test_case,label_gold,target_ident,direction,focus_words,focus_lemma,ref_case_id,ref_templ_id,templ_id,case_templ,pred_team_X,pred_team_Y,pred_model_1
0,0,derog_neg_emote_h,1,I hate women.,hateful,women,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,non-hateful,hateful,hateful
1,1,derog_neg_emote_h,2,I hate trans people.,hateful,trans people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,hateful,hateful,hateful


In [ ]:
# ... compute accuracy
func_accuracy_dict = {}
for m in models:
    func_accuracy_dict[m] = []
    for func in pd.unique(hatecheck.functionality):
        n_cases = hatecheck[hatecheck.functionality==func].shape[0]
        n_correct = hatecheck[(hatecheck.functionality==func)&(hatecheck['label_gold']==hatecheck['pred_{}'.format(m)])].shape[0]
        func_accuracy_dict[m].append('{:.1%}'.format(n_correct/n_cases))

    # convert list to series
    func_accuracy_dict[m] = pd.Series(func_accuracy_dict[m])
    func_accuracy_dict[m].name = m

# create df from dict
func_accuracy_df = pd.Series(pd.unique(hatecheck.functionality))
func_accuracy_df.name = 'functionality'

for arc_data in func_accuracy_dict:
    func_accuracy_df = pd.concat([func_accuracy_df, pd.Series(func_accuracy_dict[arc_data])], axis =1)
func_accuracy_df

,functionality,model_1
0,derog_neg_emote_h,40.7%
1,derog_neg_attrib_h,41.4%
2,derog_dehum_h,70.7%
3,derog_impl_h,18.6%
4,threat_dir_h,82.7%
5,threat_norm_h,73.6%
6,slur_h,72.2%
7,slur_homonym_nh,63.3%
8,slur_reclaimed_nh,80.2%
9,profanity_h,50.0%
